In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys 
import pandas as pd
from winterdrb.utils import load_real_and_bogus, load_extra_background
from winterdrb.paths import BASE_DATA_DIR, train_path
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    auc,
    balanced_accuracy_score,
    precision_recall_curve,
    roc_auc_score,
)
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import numpy as np
from winterdrb.plot import generate_single_page
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

import logging

import warnings 
import glob, os
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from tqdm import tqdm
import datetime
import glob, os
import fastavro
from fastavro import reader, writer
import io
import gzip
from mirar.data.utils.compress import decode_img

from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split

from scipy import ndimage
import sqlite3
import joblib

from winterrb.utils import pick_alert, make_triplet, rotate_triplet, plot_triplet

In [2]:
logger = logging.getLogger(__name__)
logging.basicConfig(level="INFO")

In [7]:
redo_train_data = False

if redo_train_data:
    real_df, bogus_df = load_real_and_bogus()
    real_df["known"], bogus_df["known"] = True, True
    extra_bogus = load_extra_background(n_images=None)
    extra_bogus["known"] = False
    bogus_df = pd.concat([bogus_df, extra_bogus], axis=0)
    real_df["class"], bogus_df["class"] = True, False
    train_df = pd.concat([real_df, bogus_df], axis=0).reset_index(drop=True)
    train_df["ztfname"] = train_df["ztfname"].replace({np.nan: None, "nan": None})
    train_df.to_parquet(train_path)

In [ ]:
train_df = pd.read_parquet(train_path)

In [28]:
REALS = train_df[train_df['class']]
BOGUS = train_df[~train_df['class']].sample(400)
BOGUS.reset_index(drop=True, inplace=True)

In [29]:
def make_triplet(row: pd.Series, normalize: bool = False) -> np.ndarray:
    """
    Make stacked triplet from alert packet
        
    Parameters:
    ----------
    alert : dict
        Alert packet
    normalize: bool
        Normalize the cutout counts; False by default

    Returns:
    ----------
    triplet: np.ndarray
        Stacked triplet of science, reference and difference cutouts
    """
    cutout_dict = dict()
    
    for cutout in ['science', 'template', 'difference']:
        cutout_zipped = row['cutout_' + cutout]

        cut_data = decode_img(cutout_zipped)

        if normalize:
            cut_data /= np.linalg.norm(cut_data)
            cut_data -= np.nanmean(cut_data)
        
        cutout_dict[cutout] = np.nan_to_num(cut_data)

    shape = np.shape(cutout_dict[cutout]) + (3,)         
    triplet = np.zeros(shape)
    triplet[:, :, 0] = cutout_dict['science']
    triplet[:, :, 1] = cutout_dict['template']
    triplet[:, :, 2] = cutout_dict['difference']
    return triplet

def rotate_triplet(triplet):
    sci, ref, diff = triplet[:,:,0], triplet[:,:,1], triplet[:,:,2]
    trip90 = np.stack((ndimage.rotate(sci, 90), ndimage.rotate(ref, 90), 
                       ndimage.rotate(diff, 90)), axis = -1)
    trip180 = np.stack((ndimage.rotate(sci, 180), ndimage.rotate(ref, 180), 
                        ndimage.rotate(diff, 180)), axis = -1)
    trip270 = np.stack((ndimage.rotate(sci, 270), ndimage.rotate(ref, 270), 
                        ndimage.rotate(diff, 270)), axis = -1)
    return trip90, trip180, trip270

In [30]:
make_triplet(train_df.iloc[0])

array([[[ 9.40258503e+00, -2.70922780e+00,  1.34585987e+01],
        [-1.57603610e+00, -9.04600441e-01,  7.43493592e-01],
        [-2.32718868e+01,  2.11288378e-01, -2.17775060e+01],
        ...,
        [ 3.19262892e-01,  8.80643654e+00, -4.75910853e+00],
        [-7.62148798e-01,  1.61137268e-01,  4.80268557e-01],
        [ 7.94740725e+00, -4.72291969e-02,  9.87890216e+00]],

       [[-7.98119593e+00, -2.60333467e+00, -3.60900093e+00],
        [ 1.54298000e+01, -1.60073006e+00,  1.86308245e+01],
        [ 1.19404230e+01, -6.75896853e-02,  1.35212212e+01],
        ...,
        [ 7.07669592e+00,  9.75986099e+00,  1.18256759e+00],
        [-1.53368979e+01,  2.56098151e+00, -1.75681883e+01],
        [-1.61597958e+01,  5.13013065e-01, -1.53052568e+01]],

       [[-1.39528875e+01, -4.86001641e-01, -1.23094679e+01],
        [-3.60746837e+00, -3.18814844e-01, -9.87285110e-01],
        [ 5.87591457e+00, -1.14580536e+00,  8.10771846e+00],
        ...,
        [-3.67479801e-01,  7.10276365e-01,

In [31]:
print(f"Number of real sources: {len(REALS)}, number of bogus sources: {len(BOGUS)}")

Number of real sources: 159, number of bogus sources: 400


In [32]:
print('Stacking reals')
cut_real = np.zeros(shape = (len(REALS)*4, 81, 81, 3))
missing_reals = []
for i, row in REALS.iterrows():
    triplet = make_triplet(row, normalize = True)
    t90, t180, t270 = rotate_triplet(triplet)
    cut_real[4*i] = triplet    
    cut_real[4*i + 1] = t90
    cut_real[4*i + 2] = t180
    cut_real[4*i + 3] = t270    
    
print('Stacking bogus')
nb = len(BOGUS) #1500
cut_bogus = np.zeros(shape = (len(BOGUS), 81, 81, 3))
missing_bogus = []
for i, row in BOGUS.iterrows():
    triplet = make_triplet(row, normalize = True)
    cut_bogus[i] = triplet    

Stacking reals


/var/folders/y5/nrs6cfxd0d99x77tn7tlscsm0000gr/T/ipykernel_22616/4151298500.py:26: RuntimeWarning: Mean of empty slice
  cut_data -= np.nanmean(cut_data)


Stacking bogus


In [36]:
labels_real = np.ones(shape = len(cut_real))
labels_bogus = np.zeros(shape = len(cut_bogus))
labels = np.concatenate((labels_real, labels_bogus), axis = 0)
cut = np.concatenate((cut_real, cut_bogus), axis = 0)

print(f"Number of real sources: {len(cut_real)}")
print(f"Number of bogus sources: {len(cut_bogus)}")
print(f"Total training samples: {len(cut)}")

Number of real sources: 636
Number of bogus sources: 400
Total training samples: 1036


# Train the model

In [38]:
test_split = 0.1
random_state = 95

# make train and test masks:
_, _, mask_train, mask_test = train_test_split(list(range(len(cut))), list(range(len(cut))), 
                                               test_size=test_split, random_state=random_state, shuffle = True)
masks = {'training': mask_train, 'test': mask_test}

In [39]:
#create training and testing samples
x_train, y_train = cut[mask_train], labels[mask_train]
x_test, y_test = cut[mask_test], labels[mask_test]

print(f"Training samples {np.shape(x_train)}, testing samples {np.shape(x_test)}")

Training samples (932, 81, 81, 3), testing samples (104, 81, 81, 3)


In [40]:
# tf.keras.backend.clear_session()

# loss = 'binary_crossentropy'
# optimizer = 'adam'
# epochs = 500
# patience = 50
# validation_split = 0.1
# class_weight = True
# batch_size = 64

# # halt training if no gain in validation accuracy over patience epochs
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)

# data_augmentation = {'horizontal_flip': False,
#                      'vertical_flip': False,
#                      'rotation_range': 0,
#                      'fill_mode': 'constant',
#                      'cval': 1e-9}

# #data generator
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=data_augmentation['horizontal_flip'],
#                                                           vertical_flip=data_augmentation['vertical_flip'],
#                                                           rotation_range=data_augmentation['rotation_range'],
#                                                           fill_mode=data_augmentation['fill_mode'],
#                                                           cval=data_augmentation['cval'],
#                                                           validation_split=validation_split)

# training_generator = datagen.flow(x_train, y_train, batch_size=batch_size, subset='training')
# validation_generator = datagen.flow(x_train, y_train, batch_size=batch_size, subset='validation')

NameError: name 'tf' is not defined